# TODO

- [WORKING] decide on plots
- [WORKING] add plotting and yield code to the `gcam_demeter_clinic` repo as a package and install:  https://github.com/JGCRI/gcam_demeter_clinic cloned here:  /home/msdadmin/repos/gcam_demeter_clinic
- [TODO] ADD IN MSD-LIVE LOGO, GCIMS, MSD and mention it in Setup

- Demeter run from updated land allocation from wrapper
- Check performance:  runtime, memory
- Finalize notebook 






# Schedule

-[5] **Greetings**

-[20] **Introductions**:  who are we, and what is this clinic about, and what will you do, and what you should take away from this, mention MSD and GCIMS

-[5-10] **Setup**: login, mention MSD-LIVE

-[30] **GCAM, gcamwrapper**:  What is GCAM, what is GCAM wrapper, ...

-[20] **Demeter**: What is Demeter, why it was created, etc.

-[40] **Demo**: Pralit talks about the context while GCAM is running...may want to discuss what is going on in GCAM


# CSDMS 2021 Annual Meeting

### CLINIC:  Thursday, May 20 11:00am - 1pm MST

## GCAM and Demeter:  A global, integrated human-Earth systems perspective to modeling land projections

Researchers and decision makers are increasingly interested in understanding the many ways in which human and Earth systems interact with one another, at scales from local (e.g., a city) to regional to global. For example, how might changes in population, income, or technology development alter crop production, energy demand, or water withdrawals? How do changes in one region's demand for energy affect energy, water, and land in other regions? This session will focus on two models – GCAM and Demeter – that provide capabilities to address these types of questions.


GCAM is an open-source, global, market equilibrium model that represents the linkages between energy, water, land, climate, and economic systems. A strength of GCAM is that it can be used to quickly explore, and quantify the uncertainty in, a large number of alternate future scenarios while accounting for multi-sector, human-Earth system dynamics. One of GCAM’s many outputs is projected land cover/use by subregion. Subregional projections provide context and can be used to understand regional land dynamics; however, Earth System Models (ESMs) generally require gridded representations of land at finer scales. Demeter, a land use and land cover disaggregation model, was created to provide this service. Demeter directly ingests land projections from GCAM and creates gridded products that match the desired resolution and land class requirements of the user.

This clinic will introduce both GCAM and Demeter at a high-level. We will also provide a hands-on walk through for a reference case so attendees can become familiar with configuring and running these two models. Our goal will be for attendees to leave the clinic with an understanding of 1) the value of capturing a global perspective when informing subregional and local analysis, 2) possibilities to conduct scenario exploration experiments that capture multi-sector/scale dynamics, 3) and a hands-on experience with GCAM and Demeter.

# Supporting packages

In [1]:
import pandas as pd

import gcamwrapper as gw

from demeter import Model

import demo_utils


## Using GCAM via `gcamwrapper`

#### Load GCAM specific info

In [3]:
# path to the exe directory where gcam-core is installed
core_exe_path = '/home/msdadmin/csdms_demo/gcam-core/exe'

# path to the xml configuration file you want to use
config_xml_file = 'configuration_reduced.xml'


#### Prepare GCAM

In [4]:
g = gw.Gcam(config_xml_file, core_exe_path)

Running GCAM model code base version 5.3 revision gcam-v5.3

Configuration file:  configuration_reduced.xml
Parsing input files...
Parsing ../input/gcamdata/xml/no_climate_model.xml scenario component.
Parsing ../input/gcamdata/xml/socioeconomics_gSSP2.xml scenario component.
Parsing ../input/gcamdata/xml/resources.xml scenario component.
Parsing ../input/gcamdata/xml/en_supply.xml scenario component.
Parsing ../input/gcamdata/xml/en_transformation.xml scenario component.
Parsing ../input/gcamdata/xml/electricity.xml scenario component.
Parsing ../input/gcamdata/xml/heat.xml scenario component.
Parsing ../input/gcamdata/xml/hydrogen.xml scenario component.
Parsing ../input/gcamdata/xml/en_distribution.xml scenario component.
Parsing ../input/gcamdata/xml/industry.xml scenario component.
Parsing ../input/gcamdata/xml/industry_incelas_gssp2.xml scenario component.
Parsing ../input/gcamdata/xml/cement.xml scenario component.
Parsing ../input/gcamdata/xml/cement_incelas_gssp2.xml scenario 

#### Run the model through its calibration years

In [5]:
final_cal_period = g.convert_year_to_period(2015)
g.run_to_period(final_cal_period)

Starting a model run. Running period 4
Model run beginning.
Period 0: 1975
Model solved with last period's prices.

Period 1: 1990
Model solved normally. Iterations period 1: 165. Total iterations: 166

Period 2: 2005
Model solved normally. Iterations period 2: 181. Total iterations: 346

Period 3: 2010
Model solved normally. Iterations period 3: 326. Total iterations: 671

Period 4: 2015
Model solved normally. Iterations period 4: 200. Total iterations: 870

All model periods solved correctly.
Model run completed.


#### Query land allocation data from the current GCAM run

In [6]:
# load query from query library
query_string = gw.get_query('land', 'land_allocation')

# query specific information
query_params = {'region': ['*'], 'leaf': ['*'], 'year': ['<=', g.get_current_year()]}

# create an output data frame containing land data
land_df = g.get_data(query_string, query_params)

land_df.head(6)


,region,child-nodes,year,land-allocation
0,Africa_Eastern,Corn_AfrCstE_IRR_hi,1975,0.009381
1,Africa_Eastern,Corn_AfrCstE_IRR_hi,1990,0.008705
2,Africa_Eastern,Corn_AfrCstE_IRR_hi,2005,0.010365
3,Africa_Eastern,Corn_AfrCstE_IRR_hi,2010,0.012348
4,Africa_Eastern,Corn_AfrCstE_IRR_hi,2015,0.013194
5,Africa_Eastern,Corn_AfrCstE_IRR_lo,1975,0.009381


# Handing off to Demeter

In [8]:
# TODO:  read directly from the `land_df` producted by gcamwrapper; right now, read the content from a
#  pickled file locally for testing
land_df = pd.read_pickle('/home/msdadmin/csdms_demo/demeter_data/inputs/projected/land_alloc.pkl')


## Run Demeter

In [7]:
config_file = '/home/msdadmin/csdms_demo/demeter_data/csdms_clinic_config.ini'

# instantiate demeter model
model = Model(config_file=config_file, 
              gcamwrapper_df=land_df,
              write_outputs=False)


2021-05-12 15:09:20,688 - demeter_runtime - INFO - Using `run_dir`:  /home/msdadmin/csdms_demo/demeter_data


In [8]:
# initialize model
model.initialize()


2021-05-12 15:09:25,290 - demeter_runtime - INFO - START
2021-05-12 15:09:25,291 - demeter_runtime - INFO - Reading allocation input files...
2021-05-12 15:09:25,315 - demeter_runtime - INFO - PERFORMANCE:  Allocation files processed in 0.022575855255126953 seconds
2021-05-12 15:09:25,319 - demeter_runtime - INFO - Preparing projected land use data...
2021-05-12 15:09:25,319 - demeter_runtime - INFO - Using projected GCAM data from `gcamwrapper` data frame


 'protectedunmanagedpasture']


2021-05-12 15:09:25,448 - demeter_runtime - INFO - Number of regions from projected file:  33


INFO:demeter_runtime:Number of regions from projected file:  33


2021-05-12 15:09:25,449 - demeter_runtime - INFO - Number of basins from projected file:  231


...

INFO:demeter_runtime:PERFORMANCE:  Constraints applied to projected and spatial data in 0.6750528812408447 seconds


2021-05-12 15:09:26,538 - demeter_runtime - INFO - Creating and processing kernel density...


INFO:demeter_runtime:Creating and processing kernel density...


2021-05-12 15:09:27,207 - demeter_runtime - INFO - PERFORMANCE:  Kernel density filter prepared in 0.667750358581543 seconds


INFO:demeter_runtime:PERFORMANCE:  Kernel density filter prepared in 0.667750358581543 seconds


In [9]:
# process first year
df_2015 = model.process_step()


2021-05-12 15:09:44,445 - demeter_runtime - INFO - Applying intensification: pass 1 for time step 2015...


INFO:demeter_runtime:Applying intensification: pass 1 for time step 2015...


2021-05-12 15:10:04,211 - demeter_runtime - INFO - Applying expansion for time step 2015...


INFO:demeter_runtime:Applying expansion for time step 2015...


2021-05-12 15:10:20,622 - demeter_runtime - INFO - Applying intensification: pass 2 for time step 2015...


INFO:demeter_runtime:Applying intensification: pass 2 for time step 2015...


In [10]:
df_2015.head()

,latitude,longitude,basin_id,region_id,water,crop1_irr,crop10_irr,crop11_irr,crop12_irr,crop13_irr,...,rockicedesert,shrubland,unmanagedforest,unmanagedpasture,urbanland,biomassgrass_irr,biomassgrass_rfd,biomasstree_irr,biomasstree_rfd,tundra
0,10.25,-0.25,160.0,5.0,0.0,0.0,0.000292,0.000292,0.000082,0.000082,...,0.0,0.000000,0.0,0.196330,0.000000,0.0,0.0,0.0,0.0,0.000000
1,10.75,-0.25,160.0,5.0,0.0,0.0,0.000069,0.000069,0.000000,0.000000,...,0.0,0.000000,0.0,0.081875,0.000000,0.0,0.0,0.0,0.0,0.000000
2,11.25,-0.25,160.0,5.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.283890,0.001667,0.0,0.0,0.0,0.0,0.017833
3,11.75,-0.25,160.0,5.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.284394,0.003472,0.0,0.0,0.0,0.0,0.060474
4,12.25,-0.25,149.0,5.0,0.0,0.0,0.000002,0.000002,0.000000,0.000000,...,0.0,0.267076,0.0,0.155609,0.003208,0.0,0.0,0.0,0.0,0.165199


In [11]:
# cleanup model
model.cleanup()


2021-05-12 15:12:30,338 - demeter_runtime - INFO - END


INFO:demeter_runtime:END


## Run the next model period in GCAM without yield impacts

In [21]:
g.run_to_period()

Starting a model run. Running period 5
Model run beginning.
Period 5: 2020
Model solved normally. Iterations period 5: 938. Total iterations: 1808

All model periods solved correctly.
Model run completed.


### Query for some results with which to compare results

In [27]:
# load query for land allocation
query_string = gw.get_query('land', 'land_allocation')
query_params = {'region': ['*'], 'leaf': ['*'], 'year': ['<=', g.get_current_year()]}
# run the query
land_df = g.get_data(query_string, query_params)

# get Corn producer prices
query_string = gw.get_query('ag', 'prices')
query_params = {'region': ['*'], 'sector': ['=', 'Corn'], 'year': ['=', g.get_current_year()]}
prod_prices = g.get_data(query_string, query_params)

# get domestic Corn prices (accounting for trade)
query_params['sector'] = ["=", 'regional corn']
prod_prices = g.get_data(query_string, query_params)

# finally get the actual yields
query_string = gw.get_query('ag', 'yield')
query_params = {'region': ['*'], 'sector': ['=', 'Corn'], 'tech': ['*'], 'year': ['=', g.get_current_year()]}
yields = g.get_data(query_string, query_params)

/usr/local/lib/python3.8/dist-packages/gcamwrapper-0.1.0-py3.8-linux-x86_64.egg/gcamwrapper/main.py:54: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_df.meta = {'units': units}


## Get the yield impacts and apply them

In [24]:
yield_scaler_df = demo_utils.get_yield_scalers(df_2015, g.get_current_year())
yield_scaler_df.head()

,region,sector,subsector,technology,year,yield_scaler
Crop1,USA,Wheat,Wheat_NelsonR,Wheat_NelsonR_IRR_hi,2020,1.193611
Crop1,USA,Wheat,Wheat_MexCstNW,Wheat_MexCstNW_IRR_hi,2020,0.765466
Crop1,USA,Wheat,Wheat_California,Wheat_California_IRR_hi,2020,0.936728
Crop1,USA,Wheat,Wheat_MissppRN,Wheat_MissppRN_IRR_hi,2020,0.923013
Crop1,USA,Wheat,Wheat_MissppRS,Wheat_MissppRS_IRR_hi,2020,0.920445


In [37]:
#yields.join(yield_scaler_df.reset_index(), on=["region", "sector", "technology", "year"], how="inner").head()
yields_new = yields.set_index(["region", "sector", "technology", "year"]).join(yield_scaler_df.set_index(["region", "sector", "technology", "year"]), how="inner").reset_index()
yields_new['yield'] *= yields_new['yield_scaler']
yields_new = yields_new.filter(["region", "sector", "technology", "year", "yield"])
yields_new.head()

,region,sector,technology,year,yield
0,Africa_Eastern,Corn,Corn_AfrCstE_IRR_hi,2020,0.127408
1,Africa_Eastern,Corn,Corn_AfrCstE_IRR_lo,2020,0.084936
2,Africa_Eastern,Corn,Corn_AfrCstE_RFD_hi,2020,0.119304
3,Africa_Eastern,Corn,Corn_AfrCstE_RFD_lo,2020,0.079536
4,Africa_Eastern,Corn,Corn_AfrCstNE_IRR_hi,2020,0.052527


## Set the new yields into the model and re-run

In [38]:
# get the same query string
query_string = gw.get_query('ag', 'yield')
# however the syntax for the query params are slightly different for set data as we
# need to explicitly tell it which match with the '+' argument and of course we do
# not give the values to compare against as those are coming from the DataFrame
query_params = {'region': ['+', '='], 'sector': ['+', '='], 'tech': ['+', '='], 'year': ['+', '=']}
g.set_data(yields_new, query_string, query_params)

# re-run 2020
g.run_to_period(g.get_current_period())



Starting a model run. Running period 5
Model run beginning.
Period 5: 2020
Model solved normally. Iterations period 5: 744. Total iterations: 2551

All model periods solved correctly.
Model run completed.


## Get the results with impacts for comparison

In [39]:
# load query for land allocation
query_string = gw.get_query('land', 'land_allocation')
query_params = {'region': ['*'], 'leaf': ['*'], 'year': ['<=', g.get_current_year()]}
# run the query
land_df_new = g.get_data(query_string, query_params)

# get Corn producer prices
query_string = gw.get_query('ag', 'prices')
query_params = {'region': ['*'], 'sector': ['=', 'Corn'], 'year': ['=', g.get_current_year()]}
prod_prices_new = g.get_data(query_string, query_params)

# get domestic Corn prices (accounting for trade)
query_params['sector'] = ["=", 'regional corn']
prod_prices_new = g.get_data(query_string, query_params)

# finally get the actual yields
query_string = gw.get_query('ag', 'yield')
query_params = {'region': ['*'], 'sector': ['=', 'Corn'], 'tech': ['*'], 'year': ['=', g.get_current_year()]}
yields_new = g.get_data(query_string, query_params)

/usr/local/lib/python3.8/dist-packages/gcamwrapper-0.1.0-py3.8-linux-x86_64.egg/gcamwrapper/main.py:54: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_df.meta = {'units': units}


## Visualizations comparing 2025 with and without crop yield update

In [15]:
# either load original run without updates or do two sims

In [40]:
yields_new.head()

,region,sector,technology,year,yield
0,Africa_Eastern,Corn,Corn_AfrCstE_IRR_hi,2020,0.127408
1,Africa_Eastern,Corn,Corn_AfrCstE_IRR_lo,2020,0.084936
2,Africa_Eastern,Corn,Corn_AfrCstE_RFD_hi,2020,0.119304
3,Africa_Eastern,Corn,Corn_AfrCstE_RFD_lo,2020,0.079536
4,Africa_Eastern,Corn,Corn_AfrCstNE_IRR_hi,2020,0.052527


In [41]:
prod_prices.head()

,region,sector,year,price
0,Africa_Eastern,regional corn,2020,0.093615
1,Africa_Northern,regional corn,2020,0.075543
2,Africa_Southern,regional corn,2020,0.070293
3,Africa_Western,regional corn,2020,0.101638
4,Argentina,regional corn,2020,0.058164


In [42]:
prod_prices_new.head()

,region,sector,year,price
0,Africa_Eastern,regional corn,2020,0.101242
1,Africa_Northern,regional corn,2020,0.073072
2,Africa_Southern,regional corn,2020,0.070003
3,Africa_Western,regional corn,2020,0.104164
4,Argentina,regional corn,2020,0.061497
